In [3]:
open System

// Define a required attribute
[<AttributeUsage(AttributeTargets.Property, AllowMultiple = false)>]
type RequiredAttribute() =
    inherit Attribute()

// Define a range attribute
[<AttributeUsage(AttributeTargets.Property, AllowMultiple = false)>]
type RangeAttribute(minValue: int, maxValue: int) =
    inherit Attribute()
    member this.MinValue = minValue
    member this.MaxValue = maxValue

type Person(name: string, age: int) =
    [<Required>]
    member val Name = name with get, set

    [<Range(1, 120)>]
    member val Age = age with get, set

open System.Reflection

// Helper function to get a custom attribute
let tryGetCustomAttribute<'T when 'T :> Attribute> (prop: PropertyInfo) =
    match prop.GetCustomAttribute(typeof<'T>, true) with
    | null -> None
    | attr -> Some (attr :?> 'T)

// Function to validate an object based on its attributes
let validate (obj: obj) =
    let objType = obj.GetType()
    let properties = objType.GetProperties()

    properties |> Array.iter (fun prop ->
        let value = prop.GetValue(obj)
        
        match tryGetCustomAttribute<RequiredAttribute>(prop) with
        | Some _ when value = "" -> 
            printfn "Property %s is required" prop.Name
        | _ -> ()

        match tryGetCustomAttribute<RangeAttribute>(prop) with
        | Some attr ->
            let intValue = value :?> int
            if intValue < attr.MinValue || intValue > attr.MaxValue then
                printfn "Property %s must be between %d and %d" prop.Name attr.MinValue attr.MaxValue
        | None -> ())

// Example usage
let person = Person("", 130)
validate person




Property Name is required
Property Age must be between 1 and 120
